1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-lieve of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import gamma

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%precision 4

tl_initial = 1000
pb_initial = 0
period = 3.052*60 #seconds

def p(time):
    return 1-2**(-time/period)

def nup(time):
    return 2**(-time/period)*np.log(2)/period
def idf(p):
    return -np.log2(1-p)*period

tl = tl_initial
pb = pb_initial
decay_hist = [[tl_initial, pb_initial]]
decay_idf = idf(np.random.random(size=(tl,)))
decay_idf.sort()
decay_idf_hist =[[tl_initial,pb_initial]]

t = 1
while(tl>0):
    p_time = p(t)
    for atom in range(tl):
        p_atom = np.random.uniform(0,1)
        if p_atom < p_time:
            tl -= 1
            pb += 1
    decay_hist.append([tl,pb])
    t+=1
    
print("first method time: {}".format(len(decay_hist)))

tl = tl_initial
pb = pb_initial
for i in range(1,int(decay_idf[decay_idf.size-1])):
    cnt = sum(1 if (x>=i-1 and x < i) else 0 for x in decay_idf)
    if(tl - cnt < 0):
        tl = 0
        pb += tl
    else:
        tl -= cnt
        pb += cnt
    decay_idf_hist.append([tl,pb])
    if (tl == 0):
        break
print("second method time: {}".format(len(decay_idf_hist)))

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(decay_hist)
plt.title('First function')
plt.xlabel('Time (s)')
plt.ylabel('Number of atoms')
plt.legend(['Tl','Pb']);

plt.subplot(122)
plt.plot(decay_idf_hist)
plt.title('Second function')
plt.xlabel('Time (s)')
plt.ylabel('Number of atoms')
plt.legend(['Tl','Pb']);
print("repeated tries show different timings. Anyway the time for the first method stays near 65s and for the second method is > 2000")

2\. **Rutherford Scattering**

The scattering angle $\theta$ of $\alpha$ particles hitting a positively charged nucleus of a Gold atom ($Z=79$) follows the rule:

$$
\tan{\frac{1}{2} \theta} = \frac{Z e^2} {2\pi \epsilon_0 E b}
$$

where $E=7.7$ MeV and $b$ beam is the impact parameter. The beam is represented by a 2D gaussian distribution with $\sigma=a_0/100$ for both coordinates ($a_0$ being the Bohr radius). Assume 1 million $\alpha$ particles are shot on the gold atom.

Computing the fraction of particles that "bounce back",i.e. those particle whose scattering angle is greater than $\pi/2$ (which set a condition on the impact parameter $b$)

In [ ]:
alpha_num =10**6
Z = 79
electron = 1.60217*(10**-19)
E = 7.7*(10**6)*electron
epsilon_0 = 8.85419*(10**-12)
a0 = 5.29177*(10**-11)

# Box-Muller method
z = np.random.rand(alpha_num)
b = np.sqrt(-2*(a0/100)**(2)*np.log(1-z))
angle = 2*np.arctan(Z*electron**2/(2*np.pi*epsilon_0*E*b))

print('Fraction {}'.format(sum(1 if a > np.pi/2 else 0 for a in angle)/alpha_num))

3\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function 

$$f(x) =\sin^2{\frac{1}{x(2-x)}}$$

* Compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one

In [ ]:
from scipy import integrate
def f(x):
    return (np.sin(1/(x*(2-x))))**2

x = np.linspace(0,2,10000)
integral = integrate.quad(f,0,2,limit=10000)[0]
print("real value: {}".format(integral))


N = 100000
c = 0
for i in range(N):
    x = 2*np.random.random() #so range [0,2]
    if np.random.random() < f(x): #y < f(x)
        c += 1
value = 2*c/N
print('Value: {} \t Error: {}'.format(value, abs(integral-value)))

mean = 2*f(2*np.random.random(N)).sum()/N
print('Value: {} \t Error: {}'.format(mean, abs(integral-mean)))

4\. **Monte Carlo integration in high dimension**

* Start of by computing the area of a circle of unit radius, by integrating the function 

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere



In [ ]:
def area(p):
    x,y = p
    if x**2+y**2<1:
        return 1
    else:
        return 0
    
N = 100000
points = 2*np.random.random(size=(N, 2))-1
tot = 0
for p in points:
    tot+=area(p)
print("found: {} \t Error:{}".format(4*tot/N, abs(4*tot/N-np.pi)))

def general(p):
    to_sum = p**2
    if to_sum.sum()<1:
        return 1
    else:
        return 0

points = 2*np.random.random(size=(N, 10))-1
tot = 0
for p in points:
    tot+=general(p)
print("found: {} \t Error:{}".format(2**10*tot/N, abs(2**10*tot/N-np.pi**5/120)))


5\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should get a result about 0.84

In [ ]:
def func(x):
    return x**(-0.5)/(np.exp(x)+1)

def w(x):
    return x**(-0.5)

def integral(N, l,h):
    sampling = integrate.quad(w,l,h)[0]
    
    res = 0.0
    x = np.random.rand(N)
    res = func(x)/w(x)
    return res.sum()*sampling/N

N = int(1e8)
print("Integral value: ",integral(N,0,1))